In [1]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
import gc
import time
gc.enable()
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import re
import requests
import folium
import branca.colormap as cm
import geopy
from tqdm import tqdm_notebook as tqdm
import json
import os
import geojson

warnings.filterwarnings('ignore')

In [2]:
%%time
# Load from preprocessed data
sd = pd.read_feather('../data/Final_Divvy_data_2018.feather')

CPU times: user 107 ms, sys: 308 ms, total: 415 ms
Wall time: 717 ms


In [8]:
sd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216478 entries, 0 to 216477
Data columns (total 58 columns):
station_id                     216478 non-null int64
month                          216478 non-null int64
day                            216478 non-null int64
total_in                       216478 non-null float64
total_out                      216478 non-null float64
year                           216478 non-null float64
lon_ave                        216478 non-null float64
lat_ave                        216478 non-null float64
dp_max                         216478 non-null float64
dp_min                         216478 non-null float64
city_Chicago                   216478 non-null float64
city_Evanston                  216478 non-null float64
city_Oak_Park                  216478 non-null float64
days_online                    216478 non-null int64
dayofweek                      216478 non-null int64
apparentTemperatureHigh        216478 non-null float64
apparentTemperature

In [3]:
def get_data_of_a_day(month, day):
    sub_df = sd[(sd['month'] == month) & (sd['day'] == day)]
    lat_arr = np.array(sub_df.lat_ave)
    lon_arr = np.array(sub_df.lon_ave)
    flg_arr = np.array(sub_df.total_in > sub_df.dp_max)
    return lat_arr, lon_arr, flg_arr

In [6]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from ipywidgets import widgets
import plotly.graph_objs as go
from plotly import tools

init_notebook_mode(connected=True)

In [7]:
## Global setup for plotting
# API key
mapbox_access_token = 'pk.eyJ1IjoibWVydnluMTUyIiwiYSI6ImNqeHpkNWZmdjAxczUzY29hbHVoandyMnUifQ.D-botzm1Hr6Gjs8jqwD5VA'

In [269]:
## Function to get map data
def _get_map_data(df, month, day, nm, hottag=True, color='orange', subplot='mapbox', 
                  op=0.5, sz=15, vis=False, sl=True):
    if hottag:
        sub_df = df[(df['month'] == month) & (df['day'] == day) & (df.total_in >= df.dp_max)]
    else:
        sub_df = df[(df['month'] == month) & (df['day'] == day) & (df.total_in < df.dp_max)]

    return go.Scattermapbox(
        lat=list(np.array(sub_df.lat_ave)),
        lon=list(np.array(sub_df.lon_ave)),
#         text=["<b>Station id</b>: {}".format(
#             int(sub_df.id),
#         )],
        mode='markers',
        hoverinfo = 'text',
        marker=go.scattermapbox.Marker(
            size=sz,
            color=color,
            opacity=op
        ),   
        subplot=subplot,
        visible=vis,
        name=nm,
        showlegend=sl,
    )

In [339]:
## Style of mixed plot
layout = {
    'title': {
        'text': 'Bike rentals for each station',
        'font': dict(
            family='Droid Serif, serif',
            size=30,
            color='Black'
        ),
    },
    'mapbox': go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=30,
        domain={'x': [0.00, 0.68], 'y': [0, 1]},
        center=go.layout.mapbox.Center(
            lat=41.878, 
            lon=-87.636
        ),
        pitch=60,
        zoom=13.5,
        style = 'mapbox://styles/mervyn152/cjy0sm1q506il1cp9gt19y457'
    ),  
    'mapbox2': go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        domain={'x': [0.72, 1], 'y': [0, 1]},
        center=go.layout.mapbox.Center(
            lat=41.898, 
            lon=-87.67
        ),
        pitch=0,
        zoom=9,
        style = 'light'
    ), 
#     'showlegend': True,
    'autosize': True,
    'legend': dict(orientation="h", x=0, y=0),
    'margin': go.layout.Margin(l=10, r=10, b=30, t=50, pad=6)
}

In [340]:
## Get data
data = []

data.append(_get_map_data(sd, 7, 1, 'Low Demand', hottag=False, color='orange', subplot='mapbox', 
                          op=1, sz=17, vis=True, sl=True))
data.append(_get_map_data(sd, 7, 1, 'Heavy Demand', hottag=True, color='red', subplot='mapbox', 
                          op=1, sz=17, vis=True, sl=True))
data.append(_get_map_data(sd, 7, 1, 'Low Demand', hottag=False, color='orange', subplot='mapbox2', 
                          op=0.3, sz=12, vis=True, sl=False))
data.append(_get_map_data(sd, 7, 1, 'Heavy Demand', hottag=True, color='red', subplot='mapbox2', 
                          op=0.5, sz=12, vis=True, sl=False))
data.append(_get_map_data(sd, 10, 29, 'Low Demand', hottag=False, color='orange', subplot='mapbox', 
                          op=1, sz=17, vis=False, sl=True))
data.append(_get_map_data(sd, 10, 29, 'Heavy Demand', hottag=True, color='red', subplot='mapbox', 
                          op=1, sz=17, vis=False, sl=True))
data.append(_get_map_data(sd, 10, 29, 'Low Demand', hottag=False, color='orange', subplot='mapbox2', 
                          op=0.3, sz=12, vis=False, sl=False))
data.append(_get_map_data(sd, 10, 29, 'Heavy Demand', hottag=True, color='red', subplot='mapbox2', 
                          op=0.5, sz=12, vis=False, sl=False))

In [341]:
updatemenus = list([
    dict(type="buttons",
         active=-1,
         buttons=list([
            dict(label = 'July 1st, 2018 (Sun)',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, False, False, False, False]}]),
            dict(label = 'Oct 29th, 2018 (Mon)',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True, True, True, True, ]}]),
        ]),
        direction = 'left',
        showactive = True,
        pad = {'r': 0, 't': 0},
        x = 1,
        xanchor = 'right',
        y = -0.1,
        yanchor = 'bottom',
#         bgcolor = 'lightgrey',
#         bordercolor = 'white',
        font = dict(size=12, color='#101010')
    ),
])

layout['updatemenus'] = updatemenus

In [342]:
figure = {}
figure['data'] = data
figure['layout'] = layout


SHOW = False

if SHOW:
    iplot(figure, config={'displayModeBar': False})
else:
    plot(figure, config={'displayModeBar': False}, filename="example_usage_2.html")

In [61]:
map_from = folium.Map(location = [41.90, -87.64], zoom_start = 11,tiles=None)
folium.TileLayer(tiles='cartodbpositron', overlay=True, max_zoom=13, min_zoom=10.5).add_to(map_from)
d1_group = folium.FeatureGroup(name="July 1st, 2018 (Sun)", overlay=False)
d2_group = folium.FeatureGroup(name="July 2nd, 2018 (Mon)", overlay=False)
d3_group = folium.FeatureGroup(name="Oct 28th, 2018 (Sun)", overlay=False)
d4_group = folium.FeatureGroup(name="Oct 29th, 2018 (Mon)", overlay=False)


lat_lst_d1, long_lst_d1, flag_d1 = get_data_of_a_day(7, 1)
lat_lst_d2, long_lst_d2, flag_d2 = get_data_of_a_day(7, 2)
lat_lst_d3, long_lst_d3, flag_d3 = get_data_of_a_day(10, 28)
lat_lst_d4, long_lst_d4, flag_d4 = get_data_of_a_day(10, 29)

rd = 8
for lat, long, f in zip(lat_lst_d1, long_lst_d1, flag_d1):
    if f:
        folium.CircleMarker([lat, long], color="red", fill=True, radius=rd, weight=0, fill_opacity=0.5).add_to(d1_group)
    else:
        folium.CircleMarker([lat, long], color="orange", fill=True, radius=rd, weight=0, fill_opacity=0.3).add_to(d1_group)
    
for lat, long, f in zip(lat_lst_d2, long_lst_d2, flag_d2):
    if f:
        folium.CircleMarker([lat, long], color="red", fill=True, radius=rd, weight=0, fill_opacity=0.5).add_to(d2_group)
    else:
        folium.CircleMarker([lat, long], color="orange", fill=True, radius=rd, weight=0, fill_opacity=0.3).add_to(d2_group)    

for lat, long, f in zip(lat_lst_d3, long_lst_d3, flag_d3):
    if f:
        folium.CircleMarker([lat, long], color="red", fill=True, radius=rd, weight=0, fill_opacity=0.5).add_to(d3_group)
    else:
        folium.CircleMarker([lat, long], color="orange", fill=True, radius=rd, weight=0, fill_opacity=0.3).add_to(d3_group) 

for lat, long, f in zip(lat_lst_d4, long_lst_d4, flag_d4):
    if f:
        folium.CircleMarker([lat, long], color="red", fill=True, radius=rd, weight=0, fill_opacity=0.5).add_to(d4_group)
    else:
        folium.CircleMarker([lat, long], color="orange", fill=True, radius=rd, weight=0, fill_opacity=0.3).add_to(d4_group) 


d1_group.add_to(map_from)
# d2_group.add_to(map_from)
# d3_group.add_to(map_from)
d4_group.add_to(map_from)


folium.LayerControl(collapsed=False).add_to(map_from)

SAVE = False

if SAVE:
    map_from.save('example_usage.html')
map_from

In [5]:
emtpy_m = folium.Map(location = [41.90, -87.64], zoom_start = 12,tiles=None)
folium.TileLayer(tiles='Stamen Watercolor', overlay=True).add_to(emtpy_m)

In [6]:
SHOW = True

if SHOW:
    emtpy_m
else:
    emtpy_m.save('empty_map.html')

In [7]:
emtpy_m